<a href="https://colab.research.google.com/github/nisha1365/Udemy_Course_4_Spark/blob/main/Section_11_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 19.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=2b8e0716b9d4ee4c5e4bd7118985bbcd1469afdf3677938971e34df357a8e9b9
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [7]:
from pyspark.ml.regression import LinearRegression

In [9]:
data = spark.read.csv('/content/Ecommerce_Customers.csv',header = True,
                      inferSchema =True )

In [10]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [11]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [12]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [13]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [17]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [18]:
assembler = VectorAssembler(inputCols=[ 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [19]:
output = assembler.transform(data)

In [21]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [22]:
final_data = output.select('features','Yearly Amount Spent')

In [23]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [24]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [25]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean|  500.8362287089391|
| stddev|  76.92137839630821|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean|   495.988488421206|
| stddev|  84.46726418178878|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [29]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [30]:
lr_model = lr.fit(train_data)

In [31]:
test_result = lr_model.evaluate(test_data)

In [32]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.061304878899136|
| -4.698543224661137|
|  6.422793989412639|
|    4.2506129136749|
|-21.876018973535224|
| -3.839829774667578|
|  2.887203889841544|
| 3.5918995221210253|
| -4.653246305535049|
| -7.951928492585353|
| -6.256946213666879|
| -4.113088741168951|
|  3.248361639749703|
| 17.065054676098327|
|-1.5034993480814478|
|-11.746049244028882|
|  8.154799375014193|
| -2.696417082824837|
|-10.562232936914938|
|-17.548280185394162|
+-------------------+
only showing top 20 rows



In [34]:
test_result.rootMeanSquaredError

9.394665557202487

In [35]:
test_result.r2

0.9875502438136443

In [36]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [37]:
unlabeled_data = test_data.select('features')

In [38]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.4925366965402...|
|[30.9716756438877...|
|[31.0472221394875...|
|[31.1239743499119...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.3584771924370...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5316044825729...|
|[31.6098395733896...|
|[31.7216523605090...|
|[31.8093003166791...|
|[31.8512531286083...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.9048571310136...|
+--------------------+
only showing top 20 rows



In [39]:
predictions = lr_model.transform(unlabeled_data)

In [41]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|398.57904619372835|
|[30.4925366965402...|287.16978894457566|
|[30.9716756438877...| 488.2158157674801|
|[31.0472221394875...| 388.2467862753465|
|[31.1239743499119...|  508.823072813301|
|[31.2681042107507...| 427.3103629484915|
|[31.3091926408918...| 429.8335139500921|
|[31.3584771924370...| 491.5840509273544|
|[31.4252268808548...|  535.419964960297|
|[31.4474464941278...|426.55467058780937|
|[31.5147378578019...| 496.0694342101283|
|[31.5171218025062...|280.03150939155466|
|[31.5316044825729...|433.26724408961286|
|[31.6098395733896...|427.48049497500983|
|[31.7216523605090...| 349.2804259799541|
|[31.8093003166791...|   548.51794860687|
|[31.8512531286083...| 464.8374472917842|
|[31.8530748017465...| 461.9815405451768|
|[31.8648325480987...| 450.4535134137286|
|[31.9048571310136...| 491.4981376082103|
+--------------------+------------